In [2]:
! pip install mysql

  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Trying pkg-config --exists mysqlclient
      Command 'pkg-config --exists mysqlclient' returned non-zero exit status 1.
      Trying pkg-config --exists mariadb
      Command 'pkg-config --exists mariadb' returned non-zero exit status 1.
      Trying pkg-config --exists libmariadb
      Command 'pkg-config --exists libmariadb' returned non-zero exit status 1.
      Traceback (most recent call last):
        File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        

In [3]:
!pip uninstall mysql -y

In [4]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 124.5 MB/s eta 0:00:0000:01


In [36]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account
import mysql.connector
from mysql.connector import Error
import io
from datetime import datetime

# GCS configuration
gcp_bucket_name = "myfirstproject-inputbucket"
gcp_credentials_path = "double-balm-438113-f5-53067ace2784.json"

# MySQL configuration
mysql_config = {
    'host': '34.95.229.146',
    'database': 'myindustry',
    'user': 'professor',
    'password': 'senha'
}

def get_gcs_client():
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    return storage.Client(credentials=credentials)

def read_parquet_from_gcs(bucket, blob_name):
    blob = bucket.blob(blob_name)
    buffer = io.BytesIO()
    blob.download_to_file(buffer)
    buffer.seek(0)
    return pq.read_table(buffer).to_pandas()

def insert_data_to_mysql(connection, df):
    cursor = connection.cursor()
    
    # Create table if not exists
    create_table_query = """
    CREATE TABLE IF NOT EXISTS event_data (
        day INT,
        month INT,
        year INT,
        tenantId VARCHAR(255),
        eventType VARCHAR(255),
        eventId VARCHAR(255),
        temperature FLOAT,
        relativeHumidity FLOAT,
        updateTime DATETIME,
        deviceId VARCHAR(255),
        projectId VARCHAR(255),
        deviceType VARCHAR(255),
        productNumber VARCHAR(255)
    )
    """
    cursor.execute(create_table_query)
    
    # Insert data
    insert_query = """
    INSERT INTO event_data (
        day, month, year, tenantId, eventType, eventId, 
        temperature, relativeHumidity, updateTime,
        deviceId, projectId, deviceType, productNumber
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    for _, row in df.iterrows():
        data = (
            row['day'], row['month'], row['year'],
            row['tenantId'], row['eventType'], row['eventId'],
            row['data']['event']['data']['humidity']['temperature'],
            row['data']['event']['data']['humidity']['relativeHumidity'],
            row['data']['event']['data']['humidity']['updateTime'],
            row['data']['metadata']['deviceId'],
            row['data']['metadata']['projectId'],
            row['data']['metadata']['deviceType'],
            row['data']['metadata']['productNumber']
        )
        cursor.execute(insert_query, data)
    
    connection.commit()
    print(f"Inserted {len(df)} rows into MySQL")

def main():
    storage_client = get_gcs_client()
    bucket = storage_client.bucket(gcp_bucket_name)
    
    connection = None
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            
            # List all blobs in the bucket
            blobs = bucket.list_blobs()
            
            for blob in blobs:
                if blob.name.endswith('.parquet'):
                    print(f"Processing file: {blob.name}")
                    df = read_parquet_from_gcs(bucket, blob.name)
                    insert_data_to_mysql(connection, df)
            
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")
    
    main()
    
    end_time = datetime.now()
    print(f"End time: {end_time}")
    print(f"Elapsed time: {end_time - start_time}")

Start time: 2024-10-18 22:41:26.287290
Connected to MySQL database
Processing file: 0grd_nhvq_.parquet
MySQL connection is closed


KeyError: 'event'

In [37]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account
import mysql.connector
from mysql.connector import Error
import io
from datetime import datetime

# GCS configuration
gcp_bucket_name = "myfirstproject-inputbucket"
gcp_credentials_path = "double-balm-438113-f5-53067ace2784.json"

# MySQL configuration
mysql_config = {
    'host': '34.95.229.146',
    'database': 'myindustry',
    'user': 'professor',
    'password': 'senha'
}

def get_gcs_client():
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    return storage.Client(credentials=credentials)

def read_parquet_from_gcs(bucket, blob_name):
    blob = bucket.blob(blob_name)
    buffer = io.BytesIO()
    blob.download_to_file(buffer)
    buffer.seek(0)
    return pq.read_table(buffer).to_pandas()

def insert_data_to_mysql(connection, df):
    cursor = connection.cursor()
    
    # Create table if not exists
    create_table_query = """
    CREATE TABLE IF NOT EXISTS event_data (
        day INT,
        month INT,
        year INT,
        tenantId VARCHAR(255),
        eventType VARCHAR(255),
        eventId VARCHAR(255),
        nome_fazenda VARCHAR(255),
        proprietario VARCHAR(255),
        cnpj_cpf VARCHAR(255),
        endereco TEXT,
        telefone VARCHAR(255),
        email VARCHAR(255),
        tipo_materia_prima VARCHAR(255),
        quantidade_recebida FLOAT,
        data_recebimento DATETIME,
        qualidade TEXT,
        nome_produto VARCHAR(255),
        categoria VARCHAR(255),
        descricao TEXT,
        unidade_medida VARCHAR(255),
        preco_unitario FLOAT,
        prazo_validade INT,
        nome_funcionario VARCHAR(255),
        cargo VARCHAR(255),
        departamento VARCHAR(255),
        data_contratacao DATETIME,
        salario FLOAT,
        contato_emergencia VARCHAR(255)
    )
    """
    cursor.execute(create_table_query)
    
    # Insert data
    insert_query = """
    INSERT INTO event_data (
        day, month, year, tenantId, eventType, eventId,
        nome_fazenda, proprietario, cnpj_cpf, endereco, telefone, email,
        tipo_materia_prima, quantidade_recebida, data_recebimento, qualidade,
        nome_produto, categoria, descricao, unidade_medida, preco_unitario, prazo_validade,
        nome_funcionario, cargo, departamento, data_contratacao, salario, contato_emergencia, 
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    for _, row in df.iterrows():
        data = (
            row['day'], row['month'], row['year'],
            row['tenantId'], row['eventType'], row['eventId'],
            row['data']['Fornecedor']['Nome_Fazenda'],
            row['data']['Fornecedor']['Proprietario'],
            row['data']['Fornecedor']['CNPJ_CPF'],
            row['data']['Fornecedor']['Endereco'],
            row['data']['Fornecedor']['Telefone'],
            row['data']['Fornecedor']['Email'],
            row['data']['MateriaPrima']['Tipo_MateriaPrima'],
            row['data']['MateriaPrima']['Quantidade_Recebida'],
            row['data']['MateriaPrima']['Data_Recebimento'],
            row['data']['MateriaPrima']['Qualidade'],
            row['data']['Produto']['Nome_Produto'],
            row['data']['Produto']['Categoria'],
            row['data']['Produto']['Descricao'],
            row['data']['Produto']['Unidade_Medida'],
            row['data']['Produto']['Preco_Unitario'],
            row['data']['Produto']['Prazo_Validade'],
            row['data']['Funcionario']['Nome'],
            row['data']['Funcionario']['Cargo'],
            row['data']['Funcionario']['Departamento'],
            row['data']['Funcionario']['Data_Contratacao'],
            row['data']['Funcionario']['Salario'],
            row['data']['Funcionario']['Contato_Emergencia']
        )
        cursor.execute(insert_query, data)
    
    connection.commit()
    print(f"Inserted {len(df)} rows into MySQL")

def main():
    storage_client = get_gcs_client()
    bucket = storage_client.bucket(gcp_bucket_name)
    
    connection = None
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            
            # List all blobs in the bucket
            blobs = bucket.list_blobs()
            
            for blob in blobs:
                if blob.name.endswith('.parquet'):
                    print(f"Processing file: {blob.name}")
                    df = read_parquet_from_gcs(bucket, blob.name)
                    insert_data_to_mysql(connection, df)
            
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")
    
    main()
    
    end_time = datetime.now()
    print(f"End time: {end_time}")
    print(f"Elapsed time: {end_time - start_time}")

Start time: 2024-10-18 23:02:56.856177
Connected to MySQL database
Processing file: 0grd_nhvq_.parquet
Inserted 1 rows into MySQL
Processing file: 3hxl_j2wz_.parquet
Inserted 1 rows into MySQL
Processing file: m53r_mayt_.parquet
Inserted 1 rows into MySQL
MySQL connection is closed
End time: 2024-10-18 23:02:58.090831
Elapsed time: 0:00:01.234654


In [8]:
!nc -zv 34.95.229.146 3306

nc: connect to 34.95.229.146 port 3306 (tcp) failed: Connection timed out


In [9]:
!curl ifconfig.me

34.95.176.129

In [6]:
!curl ifconfig.me

35.198.14.123

In [3]:
!curl ifconfig.me

34.95.223.17

In [9]:
!curl ifconfig.me

34.95.223.17

In [1]:
df = pd.read_parquet('8j9f_c4hd_.parquet')

NameError: name 'pd' is not defined

In [18]:
df

,day,month,year,tenantId,eventType,eventId,data
0,24,12,1972,8j9f,pressurize,8j9f_c4hd,"{'event': {'eventId': '8j9f_c4hd', 'targetName..."


0    {'event': {'eventId': '8j9f_c4hd', 'targetName...
Name: data, dtype: object

In [39]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account
import mysql.connector
from mysql.connector import Error
import io
from datetime import datetime

# Configurações do GCS
gcp_bucket_name = "myfirstproject-inputbucket"
gcp_credentials_path = "double-balm-438113-f5-53067ace2784.json"

# Configurações do MySQL
mysql_config = {
    'host': '34.95.229.146',
    'database': 'myindustry',
    'user': 'professor',
    'password': 'senha'
}

def get_gcs_client():
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    return storage.Client(credentials=credentials)

def read_parquet_from_gcs(bucket, blob_name):
    blob = bucket.blob(blob_name)
    buffer = io.BytesIO()
    blob.download_to_file(buffer)
    buffer.seek(0)
    return pq.read_table(buffer).to_pandas()

def insert_data_to_mysql(connection, df):
    cursor = connection.cursor()
    
    # Inserir dados na tabela existente
    insert_query = """
    INSERT INTO event_data (
        nome_fazenda, proprietario, cnpj_cpf, endereco, telefone, email,
        tipo_materia_prima, quantidade_recebida, data_recebimento, qualidade,
        nome_produto, categoria, descricao, unidade_medida, preco_unitario, prazo_validade,
        nome_funcionario, cargo, departamento, data_contratacao, salario, contato_emergencia
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    for _, row in df.iterrows():
        data = (
            row['day'], row['month'], row['year'],
            row['tenantId'], row['eventType'], row['eventId'],
            row['data']['Fornecedor']['Nome_Fazenda'],
            row['data']['Fornecedor']['Proprietario'],
            row['data']['Fornecedor']['CNPJ_CPF'],
            row['data']['Fornecedor']['Endereco'],
            row['data']['Fornecedor']['Telefone'],
            row['data']['Fornecedor']['Email'],
            row['data']['MateriaPrima']['Tipo_MateriaPrima'],
            row['data']['MateriaPrima']['Quantidade_Recebida'],
            row['data']['MateriaPrima']['Data_Recebimento'],
            row['data']['MateriaPrima']['Qualidade'],
            row['data']['Produto']['Nome_Produto'],
            row['data']['Produto']['Categoria'],
            row['data']['Produto']['Descricao'],
            row['data']['Produto']['Unidade_Medida'],
            row['data']['Produto']['Preco_Unitario'],
            row['data']['Produto']['Prazo_Validade'],
            row['data']['Funcionario']['Nome'],
            row['data']['Funcionario']['Cargo'],
            row['data']['Funcionario']['Departamento'],
            row['data']['Funcionario']['Data_Contratacao'],
            row['data']['Funcionario']['Salario'],
            row['data']['Funcionario']['Contato_Emergencia']
        )
        cursor.execute(insert_query, data)
    
    connection.commit()
    print(f"Inserted {len(df)} rows into MySQL")

def main():
    storage_client = get_gcs_client()
    bucket = storage_client.bucket(gcp_bucket_name)
    
    connection = None
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            
            # Listar todos os blobs no bucket
            blobs = bucket.list_blobs()
            
            for blob in blobs:
                if blob.name.endswith('.parquet'):
                    print(f"Processing file: {blob.name}")
                    df = read_parquet_from_gcs(bucket, blob.name)
                    insert_data_to_mysql(connection, df)
            
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")
    
    main()
    
    end_time = datetime.now()
    print(f"End time: {end_time}")
    print(f"Elapsed time: {end_time - start_time}")


Start time: 2024-10-18 23:34:31.840136
Connected to MySQL database
Processing file: 0grd_nhvq_.parquet
Error while connecting to MySQL: 1136 (21S01): Column count doesn't match value count at row 1
MySQL connection is closed
End time: 2024-10-18 23:34:33.242388
Elapsed time: 0:00:01.402252


In [41]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account
import mysql.connector
from mysql.connector import Error
import io
from datetime import datetime

# GCS configuration
gcp_bucket_name = "myfirstproject-inputbucket"
gcp_credentials_path = "double-balm-438113-f5-53067ace2784.json"

# MySQL configuration
mysql_config = {
    'host': '34.95.229.146',
    'database': 'myindustry',
    'user': 'professor',
    'password': 'senha'
}

def get_gcs_client():
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    return storage.Client(credentials=credentials)

def read_parquet_from_gcs(bucket, blob_name):
    blob = bucket.blob(blob_name)
    buffer = io.BytesIO()
    blob.download_to_file(buffer)
    buffer.seek(0)
    return pq.read_table(buffer).to_pandas()

def insert_data_to_mysql(connection, df):
    try:
        cursor = connection.cursor()

        # Criação das tabelas
        create_tables_queries = [
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`fornecedor` (
                `cnpj` VARCHAR(20) NOT NULL,
                `nome` VARCHAR(45) NULL,
                `email` VARCHAR(45) NULL,
                `telefone` VARCHAR(45) NULL,
                PRIMARY KEY (`cnpj`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`materiaPrima` (
                `id` INT NOT NULL AUTO_INCREMENT,
                `data_recebimento` DATE NULL,
                `quantidade` INT NULL,
                `tipo` VARCHAR(45) NULL,
                `descricao` MEDIUMTEXT NULL,
                PRIMARY KEY (`id`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`setor` (
                `idsetor` INT NOT NULL AUTO_INCREMENT,
                `quantidade_produzida` INT NULL,
                PRIMARY KEY (`idsetor`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`funcionario` (
                `idfuncionario` INT NOT NULL AUTO_INCREMENT,
                `cargo` VARCHAR(45) NULL,
                `nome` VARCHAR(45) NULL,
                `data_contratacao` DATE NULL,
                `contato` VARCHAR(20) NULL,
                `salario` FLOAT NULL,
                `setor_idsetor` INT NOT NULL,
                PRIMARY KEY (`idfuncionario`),
                INDEX `fk_funcionario_setor1_idx` (`setor_idsetor` ASC),
                CONSTRAINT `fk_funcionario_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`estoque` (
                `idestoque` INT NOT NULL AUTO_INCREMENT,
                `quantidade_disponivel` INT NULL,
                PRIMARY KEY (`idestoque`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`produto` (
                `idproduto` INT NOT NULL AUTO_INCREMENT,
                `nome_produto` VARCHAR(45) NULL,
                `categoria` VARCHAR(45) NULL,
                `validade_lote` DATE NULL,
                `setor_idsetor` INT NOT NULL,
                `estoque_idestoque` INT NOT NULL,
                PRIMARY KEY (`idproduto`),
                INDEX `fk_produto_setor1_idx` (`setor_idsetor` ASC),
                INDEX `fk_produto_estoque1_idx` (`estoque_idestoque` ASC),
                CONSTRAINT `fk_produto_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_produto_estoque1`
                    FOREIGN KEY (`estoque_idestoque`)
                    REFERENCES `myindustry`.`estoque` (`idestoque`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`equipamento` (
                `IDequipamento` INT NOT NULL AUTO_INCREMENT,
                `tipo` VARCHAR(45) NULL,
                `data_aquisicao` DATE NULL,
                `ultima_manutencao` DATE NULL,
                `status` ENUM('ativo', 'inativo', 'manutenção') NULL,
                `setor_idsetor` INT NOT NULL,
                PRIMARY KEY (`IDequipamento`),
                INDEX `fk_equipamento_setor1_idx` (`setor_idsetor` ASC),
                CONSTRAINT `fk_equipamento_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`manutencao` (
                `idmanutencao` INT NOT NULL AUTO_INCREMENT,
                `data_manutencao` DATE NULL,
                `equipamento_IDequipamento` INT NOT NULL,
                `funcionario_idfuncionario` INT NOT NULL,
                PRIMARY KEY (`idmanuntencao`),
                INDEX `fk_manuntencao_equipamento1_idx` (`equipamento_IDequipamento` ASC),
                INDEX `fk_manuntencao_funcionario1_idx` (`funcionario_idfuncionario` ASC),
                CONSTRAINT `fk_manuntencao_equipamento1`
                    FOREIGN KEY (`equipamento_IDequipamento`)
                    REFERENCES `myindustry`.`equipamento` (`IDequipamento`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_manuntencao_funcionario1`
                    FOREIGN KEY (`funcionario_idfuncionario`)
                    REFERENCES `myindustry`.`funcionario` (`idfuncionario`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`fornecedor_has_materiaPrima` (
                `fornecedor_cnpj` VARCHAR(20) NOT NULL,
                `materiaPrima_id` INT NOT NULL,
                PRIMARY KEY (`fornecedor_cnpj`, `materiaPrima_id`),
                INDEX `fk_fornecedor_has_materiaPrima_materiaPrima1_idx` (`materiaPrima_id` ASC),
                INDEX `fk_fornecedor_has_materiaPrima_fornecedor_idx` (`fornecedor_cnpj` ASC),
                CONSTRAINT `fk_fornecedor_has_materiaPrima_fornecedor`
                    FOREIGN KEY (`fornecedor_cnpj`)
                    REFERENCES `myindustry`.`fornecedor` (`cnpj`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_fornecedor_has_materiaPrima_materiaPrima1`
                    FOREIGN KEY (`materiaPrima_id`)
                    REFERENCES `myindustry`.`materiaPrima` (`id`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`materiaPrima_has_setor` (
                `materiaPrima_id` INT NOT NULL,
                `setor_idsetor` INT NOT NULL,
                PRIMARY KEY (`materiaPrima_id`, `setor_idsetor`),
                INDEX `fk_materiaPrima_has_setor_setor1_idx` (`setor_idsetor` ASC),
                INDEX `fk_materiaPrima_has_setor_materiaPrima1_idx` (`materiaPrima_id` ASC),
                CONSTRAINT `fk_materiaPrima_has_setor_materiaPrima1`
                    FOREIGN KEY (`materiaPrima_id`)
                    REFERENCES `myindustry`.`materiaPrima` (`id`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_materiaPrima_has_setor_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """
        ]

        for query in create_tables_queries:
            cursor.execute(query)

        # Inserção de dados
        for _, row in df.iterrows():
            # Inserir Fornecedor
            insert_fornecedor_query = """
            INSERT INTO Fornecedores (Nome_Fazenda, Proprietario, CNPJ_CPF, Endereco, Telefone, Email)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            fornecedor_data = (
                row['data']['Fornecedor']['Nome_Fazenda'],
                row['data']['Fornecedor']['Proprietario'],
                row['data']['Fornecedor']['CNPJ_CPF'],
                row['data']['Fornecedor']['Endereco'],
                row['data']['Fornecedor']['Telefone'],
                row['data']['Fornecedor']['Email']
            )
            cursor.execute(insert_fornecedor_query, fornecedor_data)
            id_fornecedor = cursor.lastrowid

            # Inserir Materia_Prima
            insert_materia_prima_query = """
            INSERT INTO Materia_Prima (ID_Fornecedor, Tipo_MP, Quantidade, Data_Recebimento, Qualidade)
            VALUES (%s, %s, %s, %s, %s)
            """
            materia_prima_data = (
                id_fornecedor,
                row['data']['MateriaPrima']['Tipo_MateriaPrima'],
                row['data']['MateriaPrima']['Quantidade_Recebida'],
                row['data']['MateriaPrima']['Data_Recebimento'],
                row['data']['MateriaPrima']['Qualidade']
            )
            cursor.execute(insert_materia_prima_query, materia_prima_data)

            # Inserir Produto
            insert_produto_query = """
            INSERT INTO Produtos (Nome_Produto, Categoria, Descricao, Unidade_Medida, Preco_Unitario, Prazo_Validade)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            produto_data = (
                row['data']['Produto']['Nome_Produto'],
                row['data']['Produto']['Categoria'],
                row['data']['Produto']['Descricao'],
                row['data']['Produto']['Unidade_Medida'],
                row['data']['Produto']['Preco_Unitario'],
                row['data']['Produto']['Prazo_Validade']
            )
            cursor.execute(insert_produto_query, produto_data)

            # Inserir Funcionario
            insert_funcionario_query = """
            INSERT INTO Funcionarios (Nome, Cargo, Departamento, Data_Contratacao, Salario, Contato_Emergencia)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            funcionario_data = (
                row['data']['Funcionario']['Nome'],
                row['data']['Funcionario']['Cargo'],
                row['data']['Funcionario']['Departamento'],
                row['data']['Funcionario']['Data_Contratacao'],
                row['data']['Funcionario']['Salario'],
                row['data']['Funcionario']['Contato_Emergencia']
            )
            cursor.execute(insert_funcionario_query, funcionario_data)

        connection.commit()
        print(f"Inserted {len(df)} rows into MySQL tables")

    except Error as e:
        print(f"Error: {e}")
        connection.rollback()

    finally:
        if connection.is_connected():
            cursor.close()

def main():
    storage_client = get_gcs_client()
    bucket = storage_client.bucket(gcp_bucket_name)
    
    connection = None
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            
            # List all blobs in the bucket
            blobs = bucket.list_blobs()
            
            for blob in blobs:
                if blob.name.endswith('.parquet'):
                    print(f"Processing file: {blob.name}")
                    df = read_parquet_from_gcs(bucket, blob.name)
                    insert_data_to_mysql(connection, df)
            
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")
    
    main()
    
    end_time = datetime.now()
    print(f"End time: {end_time}")
    print(f"Elapsed time: {end_time - start_time}")

Start time: 2024-10-19 20:20:09.942498
Connected to MySQL database
Processing file: ljk9_wq8d_.parquet
Error: 1072 (42000): Key column 'idmanuntencao' doesn't exist in table
Processing file: u4f0_uzpr_.parquet
Error: 1072 (42000): Key column 'idmanuntencao' doesn't exist in table
Processing file: u4f0_zqtt_.parquet
Error: 1072 (42000): Key column 'idmanuntencao' doesn't exist in table
MySQL connection is closed
End time: 2024-10-19 20:20:12.380751
Elapsed time: 0:00:02.438253


In [22]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account
import mysql.connector
from mysql.connector import Error
import io
from datetime import datetime

# GCS configuration
gcp_bucket_name = "myfirstproject-inputbucket"
gcp_credentials_path = "double-balm-438113-f5-53067ace2784.json"

# MySQL configuration
mysql_config = {
    'host': '34.95.229.146',
    'database': 'myindustry',
    'user': 'professor',
    'password': 'senha'
}

def get_gcs_client():
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    return storage.Client(credentials=credentials)

def read_parquet_from_gcs(bucket, blob_name):
    blob = bucket.blob(blob_name)
    buffer = io.BytesIO()
    blob.download_to_file(buffer)
    buffer.seek(0)
    return pq.read_table(buffer).to_pandas()

def create_tables(connection):
    try:
        cursor = connection.cursor()

        # Criação das tabelas
        create_tables_queries = [
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`fornecedor` (
                `cnpj` VARCHAR(20) NOT NULL,
                `nome` VARCHAR(45) NULL,
                `email` VARCHAR(45) NULL,
                `telefone` VARCHAR(45) NULL,
                PRIMARY KEY (`cnpj`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`mateiraPrima` (
                `id` INT NOT NULL AUTO_INCREMENT,
                `data_recebimento` DATE NULL,
                `quantidade` INT NULL,
                `tipo` VARCHAR(45) NULL,
                `descricao` MEDIUMTEXT NULL,
                PRIMARY KEY (`id`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`setor` (
                `idsetor` INT NOT NULL AUTO_INCREMENT,
                `quantidade_produzida` INT NULL,
                PRIMARY KEY (`idsetor`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`funcionario` (
                `idfuncionario` INT NOT NULL AUTO_INCREMENT,
                `cargo` VARCHAR(45) NULL,
                `nome` VARCHAR(45) NULL,
                `data_contratacao` DATE NULL,
                `contato` VARCHAR(20) NULL,
                `salario` FLOAT NULL,
                `setor_idsetor` INT NOT NULL,
                PRIMARY KEY (`idfuncionario`),
                INDEX `fk_funcionario_setor1_idx` (`setor_idsetor` ASC),
                CONSTRAINT `fk_funcionario_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`estoque` (
                `idestoque` INT NOT NULL AUTO_INCREMENT,
                `quantidade_disponivel` INT NULL,
                PRIMARY KEY (`idestoque`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`produto` (
                `idproduto` INT NOT NULL AUTO_INCREMENT,
                `nome_produto` VARCHAR(45) NULL,
                `categoria` VARCHAR(45) NULL,
                `validade_lote` DATE NULL,
                `setor_idsetor` INT NOT NULL,
                `estoque_idestoque` INT NOT NULL,
                PRIMARY KEY (`idproduto`),
                INDEX `fk_produto_setor1_idx` (`setor_idsetor` ASC),
                INDEX `fk_produto_estoque1_idx` (`estoque_idestoque` ASC),
                CONSTRAINT `fk_produto_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_produto_estoque1`
                    FOREIGN KEY (`estoque_idestoque`)
                    REFERENCES `myindustry`.`estoque` (`idestoque`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`equipamento` (
                `IDequipamento` INT NOT NULL AUTO_INCREMENT,
                `tipo` VARCHAR(45) NULL,
                `data_aquisicao` DATE NULL,
                `ultima_manuntencao` DATE NULL,
                `status` ENUM('ativo', 'inativo', 'manutenção') NULL,
                `setor_idsetor` INT NOT NULL,
                PRIMARY KEY (`IDequipamento`),
                INDEX `fk_equipamento_setor1_idx` (`setor_idsetor` ASC),
                CONSTRAINT `fk_equipamento_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`manuntencao` (
                `idmanuntencao` INT NOT NULL AUTO_INCREMENT,
                `data_manutencao` DATE NULL,
                `equipamento_IDequipamento` INT NOT NULL,
                `funcionario_idfuncionario` INT NOT NULL,
                PRIMARY KEY (`idmanuntencao`),
                INDEX `fk_manuntencao_equipamento1_idx` (`equipamento_IDequipamento` ASC),
                INDEX `fk_manuntencao_funcionario1_idx` (`funcionario_idfuncionario` ASC),
                CONSTRAINT `fk_manuntencao_equipamento1`
                    FOREIGN KEY (`equipamento_IDequipamento`)
                    REFERENCES `myindustry`.`equipamento` (`IDequipamento`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_manuntencao_funcionario1`
                    FOREIGN KEY (`funcionario_idfuncionario`)
                    REFERENCES `myindustry`.`funcionario` (`idfuncionario`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`fornecedor_has_mateiraPrima` (
                `fornecedor_cnpj` VARCHAR(20) NOT NULL,
                `mateiraPrima_id` INT NOT NULL,
                PRIMARY KEY (`fornecedor_cnpj`, `mateiraPrima_id`),
                INDEX `fk_fornecedor_has_mateiraPrima_mateiraPrima1_idx` (`mateiraPrima_id` ASC),
                INDEX `fk_fornecedor_has_mateiraPrima_fornecedor_idx` (`fornecedor_cnpj` ASC),
                CONSTRAINT `fk_fornecedor_has_mateiraPrima_fornecedor`
                    FOREIGN KEY (`fornecedor_cnpj`)
                    REFERENCES `myindustry`.`fornecedor` (`cnpj`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_fornecedor_has_mateiraPrima_mateiraPrima1`
                    FOREIGN KEY (`mateiraPrima_id`)
                    REFERENCES `myindustry`.`mateiraPrima` (`id`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`mateiraPrima_has_setor` (
                `mateiraPrima_id` INT NOT NULL,
                `setor_idsetor` INT NOT NULL,
                PRIMARY KEY (`mateiraPrima_id`, `setor_idsetor`),
                INDEX `fk_mateiraPrima_has_setor_setor1_idx` (`setor_idsetor` ASC),
                INDEX `fk_mateiraPrima_has_setor_mateiraPrima1_idx` (`mateiraPrima_id` ASC),
                CONSTRAINT `fk_mateiraPrima_has_setor_mateiraPrima1`
                    FOREIGN KEY (`mateiraPrima_id`)
                    REFERENCES `myindustry`.`mateiraPrima` (`id`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_mateiraPrima_has_setor_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """
        ]

        for query in create_tables_queries:
            cursor.execute(query)

        connection.commit()
        print("Tables created successfully")

    except Error as e:
        print(f"Error creating tables: {e}")
        connection.rollback()

    finally:
        if connection.is_connected():
            cursor.close()

from datetime import datetime

def insert_default_setor(connection):
    try:
        cursor = connection.cursor()
        insert_setor_query = """
        INSERT INTO setor (quantidade_produzida)
        VALUES (%s)
        """
        setor_data = (100,)  # Quantidade produzida padrão
        cursor.execute(insert_setor_query, setor_data)
        connection.commit()
        print("Default setor inserted successfully")
    except Error as e:
        print(f"Error inserting default setor: {e}")
        connection.rollback()
    finally:
        if connection.is_connected():
            cursor.close()

def insert_data_to_mysql(connection, df):
    try:
        cursor = connection.cursor()

        # Convert timestamp columns to datetime
        date_columns = ['Data_Recebimento', 'Data_Contratacao']
        for col in date_columns:
            df[col] = pd.to_datetime(df[col]).dt.date

        # Inserção de dados
        for _, row in df.iterrows():
            # Inserir Fornecedor
            insert_fornecedor_query = """
            INSERT INTO fornecedor (cnpj, nome, email, telefone)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            nome = VALUES(nome),
            email = VALUES(email),
            telefone = VALUES(telefone)
            """
            fornecedor_data = (
                row['CNPJ_CPF'],
                row['Nome_Fazenda'],
                row['Email'],
                row['Telefone']
            )
            cursor.execute(insert_fornecedor_query, fornecedor_data)

            # Inserir Materia_Prima
            insert_materia_prima_query = """
            INSERT INTO mateiraPrima (data_recebimento, quantidade, tipo, descricao)
            VALUES (%s, %s, %s, %s)
            """
            materia_prima_data = (
                row['Data_Recebimento'],
                row['Quantidade_Recebida'],
                row['Tipo_MateriaPrima'],
                'Descrição da matéria prima'  # Você pode ajustar isso conforme necessário
            )
            cursor.execute(insert_materia_prima_query, materia_prima_data)

            # Inserir Produto
            insert_produto_query = """
            INSERT INTO produto (nome_produto, categoria, validade_lote, setor_idsetor, estoque_idestoque)
            VALUES (%s, %s, %s, %s, %s)
            """
            validade_lote = row['Data_Recebimento'] + pd.Timedelta(days=int(row['Prazo_Validade']))
            produto_data = (
                row['Nome_Produto'],
                row['Categoria'],
                validade_lote,  # Remova o .date()
                1,  # setor_idsetor (você precisa ajustar isso)
                1   # estoque_idestoque (você precisa ajustar isso)
            )

            cursor.execute(insert_produto_query, produto_data)

            # Inserir Funcionario
            insert_funcionario_query = """
            INSERT INTO funcionario (cargo, nome, data_contratacao, contato, salario, setor_idsetor)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            funcionario_data = (
                row['Cargo'],
                row['Nome_Funcionario'],
                row['Data_Contratacao'],
                row['Contato_Emergencia'].split(',')[1].strip(),  # Assumindo que o contato de emergência é o segundo item
                row['Salario'],
                1  # setor_idsetor (você precisa ajustar isso)
            )
            cursor.execute(insert_funcionario_query, funcionario_data)

        connection.commit()
        print(f"Inserted data from {len(df)} rows into MySQL tables")

    except Error as e:
        print(f"Error inserting data: {e}")
        connection.rollback()

    finally:
        if connection.is_connected():
            cursor.close()
def main():
    storage_client = get_gcs_client()
    bucket = storage_client.bucket(gcp_bucket_name)
    
    connection = None
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            
            # Criar tabelas
            create_tables(connection)
            insert_default_setor(connection)
            
            # List all blobs in the bucket
            blobs = bucket.list_blobs()
            
            for blob in blobs:
                if blob.name.endswith('.parquet'):
                    print(f"Processing file: {blob.name}")
                    df = read_parquet_from_gcs(bucket, blob.name)
                    insert_data_to_mysql(connection, df)
            
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")
    
    main()
    
    end_time = datetime.now()
    print(f"End time: {end_time}")
    print(f"Elapsed time: {end_time - start_time}")

Start time: 2024-10-19 16:29:16.712803
Connected to MySQL database
Tables created successfully
Default setor inserted successfully
Processing file: cp7m_cuhc_.parquet
Error inserting data: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`myindustry`.`produto`, CONSTRAINT `fk_produto_estoque1` FOREIGN KEY (`estoque_idestoque`) REFERENCES `estoque` (`idestoque`))
Processing file: cp7m_uenn_.parquet
Error inserting data: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`myindustry`.`produto`, CONSTRAINT `fk_produto_estoque1` FOREIGN KEY (`estoque_idestoque`) REFERENCES `estoque` (`idestoque`))
Processing file: vpwa_gd18_.parquet
Error inserting data: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`myindustry`.`produto`, CONSTRAINT `fk_produto_estoque1` FOREIGN KEY (`estoque_idestoque`) REFERENCES `estoque` (`idestoque`))
MySQL connection is closed
End time: 2024-10-19 16:29:19.192047
Elapsed ti

In [26]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account
import mysql.connector
from mysql.connector import Error
import io
from datetime import datetime

# GCS configuration
gcp_bucket_name = "myfirstproject-inputbucket"
gcp_credentials_path = "double-balm-438113-f5-53067ace2784.json"

# MySQL configuration
mysql_config = {
    'host': '34.95.229.146',
    'database': 'myindustry',
    'user': 'professor',
    'password': 'senha'
}

def get_gcs_client():
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    return storage.Client(credentials=credentials)

def read_parquet_from_gcs(bucket, blob_name):
    blob = bucket.blob(blob_name)
    buffer = io.BytesIO()
    blob.download_to_file(buffer)
    buffer.seek(0)
    return pq.read_table(buffer).to_pandas()

def create_tables(connection):
    try:
        cursor = connection.cursor()

        # Criação das tabelas
        create_tables_queries = [
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`mateiraPrima` (
                `id` INT NOT NULL AUTO_INCREMENT,
                `data_recebimento` DATE NULL,
                `quantidade` INT NULL,
                `tipo` VARCHAR(45) NULL,
                `descricao` MEDIUMTEXT NULL,
                PRIMARY KEY (`id`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`setor` (
                `idsetor` INT NOT NULL AUTO_INCREMENT,
                `quantidade_produzida` INT NULL,
                PRIMARY KEY (`idsetor`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`funcionario` (
                `idfuncionario` INT NOT NULL AUTO_INCREMENT,
                `cargo` VARCHAR(45) NULL,
                `nome` VARCHAR(45) NULL,
                `data_contratacao` DATE NULL,
                `contato` VARCHAR(100) NULL,
                `salario` FLOAT NULL,
                `setor_idsetor` INT NOT NULL,
                PRIMARY KEY (`idfuncionario`),
                INDEX `fk_funcionario_setor1_idx` (`setor_idsetor` ASC),
                CONSTRAINT `fk_funcionario_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`estoque` (
                `idestoque` INT NOT NULL AUTO_INCREMENT,
                `quantidade_disponivel` INT NULL,
                PRIMARY KEY (`idestoque`)
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`produto` (
                `idproduto` INT NOT NULL AUTO_INCREMENT,
                `nome_produto` VARCHAR(45) NULL,
                `categoria` VARCHAR(45) NULL,
                `validade_lote` DATE NULL,
                `setor_idsetor` INT NOT NULL,
                `estoque_idestoque` INT NOT NULL,
                PRIMARY KEY (`idproduto`),
                INDEX `fk_produto_setor1_idx` (`setor_idsetor` ASC),
                INDEX `fk_produto_estoque1_idx` (`estoque_idestoque` ASC),
                CONSTRAINT `fk_produto_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_produto_estoque1`
                    FOREIGN KEY (`estoque_idestoque`)
                    REFERENCES `myindustry`.`estoque` (`idestoque`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`equipamento` (
                `IDequipamento` INT NOT NULL AUTO_INCREMENT,
                `tipo` VARCHAR(45) NULL,
                `data_aquisicao` DATE NULL,
                `ultima_manuntencao` DATE NULL,
                `status` ENUM("quebrado", "alerta",'ativo', 'inativo', 'manutenção') NULL,
                `setor_idsetor` INT NOT NULL,
                PRIMARY KEY (`IDequipamento`),
                INDEX `fk_equipamento_setor1_idx` (`setor_idsetor` ASC),
                CONSTRAINT `fk_equipamento_setor1`
                    FOREIGN KEY (`setor_idsetor`)
                    REFERENCES `myindustry`.`setor` (`idsetor`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
            """
            CREATE TABLE IF NOT EXISTS `myindustry`.`manuntencao` (
                `idmanuntencao` INT NOT NULL AUTO_INCREMENT,
                `data_manutencao` DATE NULL,
                `equipamento_IDequipamento` INT NOT NULL,
                `funcionario_idfuncionario` INT NOT NULL,
                PRIMARY KEY (`idmanuntencao`),
                INDEX `fk_manuntencao_equipamento1_idx` (`equipamento_IDequipamento` ASC),
                INDEX `fk_manuntencao_funcionario1_idx` (`funcionario_idfuncionario` ASC),
                CONSTRAINT `fk_manuntencao_equipamento1`
                    FOREIGN KEY (`equipamento_IDequipamento`)
                    REFERENCES `myindustry`.`equipamento` (`IDequipamento`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION,
                CONSTRAINT `fk_manuntencao_funcionario1`
                    FOREIGN KEY (`funcionario_idfuncionario`)
                    REFERENCES `myindustry`.`funcionario` (`idfuncionario`)
                    ON DELETE NO ACTION
                    ON UPDATE NO ACTION
            ) ENGINE = InnoDB;
            """,
        ]

        for query in create_tables_queries:
            cursor.execute(query)

        connection.commit()
        print("Tables created successfully")

    except Error as e:
        print(f"Error creating tables: {e}")
        connection.rollback()

    finally:
        if connection.is_connected():
            cursor.close()

from datetime import datetime

def insert_default_setor(connection):
    try:
        cursor = connection.cursor()
        insert_setor_query = """
        INSERT INTO setor (quantidade_produzida)
        VALUES (%s)
        """
        setor_data = (100,)  # Quantidade produzida padrão
        cursor.execute(insert_setor_query, setor_data)
        connection.commit()
        print("Default setor inserted successfully")
    except Error as e:
        print(f"Error inserting default setor: {e}")
        connection.rollback()
    finally:
        if connection.is_connected():
            cursor.close()

def insert_data_to_mysql(connection, df):
    try:
        cursor = connection.cursor()

        # Convert timestamp columns to datetime
        date_columns = ['Data_Recebimento', 'Data_Contratacao']
        for col in date_columns:
            df[col] = pd.to_datetime(df[col]).dt.date

        # Iterar sobre as linhas do DataFrame para inserir dados
        for _, row in df.iterrows():
             
            # Inserir Estoque
            insert_estoque_query = """
            INSERT INTO estoque (quantidade_disponivel)
            VALUES (%s)
            """
            estoque_data = (
                row['quantidade_estoque'],  # Aqui assumimos que você tem uma coluna 'quantidade_estoque' no Parquet
            )
            cursor.execute(insert_estoque_query, estoque_data)
            
            # Obtenha o último ID do estoque inserido
            estoque_id = cursor.lastrowid
            
            # Inserir Materia_Prima
            insert_materia_prima_query = """
            INSERT INTO mateiraPrima (data_recebimento, quantidade, tipo, descricao)
            VALUES (%s, %s, %s, %s)
            """
            materia_prima_data = (
                row['Data_Recebimento'],
                row['Quantidade_Recebida'],
                row['Tipo_MateriaPrima'],
                'Descrição da matéria prima'
            )
            cursor.execute(insert_materia_prima_query, materia_prima_data)

            # Inserir Produto com o estoque_id correspondente
            insert_produto_query = """
            INSERT INTO produto (nome_produto, categoria, validade_lote, setor_idsetor, estoque_idestoque)
            VALUES (%s, %s, %s, %s, %s)
            """
            validade_lote = row['Data_Recebimento'] + pd.Timedelta(days=int(row['Prazo_Validade']))
            produto_data = (
                row['Nome_Produto'],
                row['Categoria'],
                validade_lote,
                1,  # setor_idsetor (você pode ajustar isso conforme necessário)
                estoque_id  # Use o estoque_id que acabamos de inserir
            )
            cursor.execute(insert_produto_query, produto_data)
            
            # Inserir Funcionario
            insert_funcionario_query = """
            INSERT INTO funcionario (cargo, nome, data_contratacao, contato, salario, setor_idsetor)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            funcionario_data = (
                row['Cargo'],
                row['Nome_Funcionario'],
                row['Data_Contratacao'],
                row['Contato_Emergencia'].split(',')[1].strip(),
                row['Salario'],
                1  # setor_idsetor (você pode ajustar isso)
            )
            cursor.execute(insert_funcionario_query, funcionario_data)

        connection.commit()
        print(f"Inserted data from {len(df)} rows into MySQL tables")

    except Error as e:
        print(f"Error inserting data: {e}")
        connection.rollback()

    finally:
        if connection.is_connected():
            cursor.close()
def main():
    storage_client = get_gcs_client()
    bucket = storage_client.bucket(gcp_bucket_name)
    
    connection = None
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            
            # Criar tabelas
            create_tables(connection)
            insert_default_setor(connection)
            
            # List all blobs in the bucket
            blobs = bucket.list_blobs()
            
            for blob in blobs:
                if blob.name.endswith('.parquet'):
                    print(f"Processing file: {blob.name}")
                    df = read_parquet_from_gcs(bucket, blob.name)
                    insert_data_to_mysql(connection, df)
            
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")
    
    main()
    
    end_time = datetime.now()
    print(f"End time: {end_time}")
    print(f"Elapsed time: {end_time - start_time}")

Start time: 2024-10-19 16:56:13.196939
Connected to MySQL database
Tables created successfully
Default setor inserted successfully
Processing file: goya_df1z_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: ks2w_np7w_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: ks2w_tn9c_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: oxl6_11u9_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: oxl6_dwp7_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: wph0_3jvb_.parquet
Inserted data from 1 rows into MySQL tables
MySQL connection is closed
End time: 2024-10-19 16:56:17.781883
Elapsed time: 0:00:04.584944


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account
import mysql.connector
from mysql.connector import Error
import io
from datetime import datetime

# GCS configuration
gcp_bucket_name = "myfirstproject-inputbucket"
gcp_credentials_path = "double-balm-438113-f5-53067ace2784.json"

# MySQL configuration
mysql_config = {
    'host': '34.95.229.146',
    'database': 'myindustry',
    'user': 'professor',
    'password': 'senha'
}

def get_gcs_client():
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    return storage.Client(credentials=credentials)

def read_parquet_from_gcs(bucket, blob_name):
    blob = bucket.blob(blob_name)
    buffer = io.BytesIO()
    blob.download_to_file(buffer)
    buffer.seek(0)
    return pq.read_table(buffer).to_pandas()

def create_tables(connection):
    try:
        cursor = connection.cursor()

        # Criação das tabelas
        create_tables_queries = [
                # 1. Setor (não tem dependências)
                 """
                CREATE TABLE IF NOT EXISTS `myindustry`.`setor` (
                
                `nome_setor` VARCHAR(45) NOT NULL,
                `quantidade_produzida` INT DEFAULT 0 
                );
                """,

                # 2. Equipamento (depende de Setor)
                """
                CREATE TABLE IF NOT EXISTS `myindustry`.`equipamento` (
                `id_equipamento` VARCHAR(7) NOT NULL,
                `nome_equipamento` VARCHAR(100) NOT NULL,
                `data_aquisicao` DATETIME NOT NULL,
                `eqp_status` ENUM('ativo', 'quebrado', 'alerta', 'inativo', 'manutenção') NOT NULL,
                `eventType` VARCHAR(20),
                
                PRIMARY KEY (`id_equipamento`)
                
                );
                """,

                # 3. Produto (depende de Equipamento)
                """
                CREATE TABLE IF NOT EXISTS `myindustry`.`produto` (
                `id_produto` INT NOT NULL AUTO_INCREMENT,
                `id_equipamento` VARCHAR(7) NOT NULL,
                `nome_produto` VARCHAR(45) NOT NULL,
                `categoria` VARCHAR(45) NOT NULL,
                `descricao` TEXT,
                `unidade_medida` VARCHAR(20) NOT NULL,
                `preco_unitario` DECIMAL(10, 2) NOT NULL,
                `prazo_validade` INT NOT NULL,
                PRIMARY KEY (`id_produto`),
                FOREIGN KEY (`id_equipamento`) REFERENCES `equipamento` (`id_equipamento`)
                );
                """,

                # 4. Estoque (depende de Produto)
                """
                CREATE TABLE IF NOT EXISTS `myindustry`.`estoque` (
                `id_estoque` INT NOT NULL AUTO_INCREMENT,
                `id_produto` INT NOT NULL,
                `quantidade_estoque` DECIMAL(10, 2) NOT NULL,
                PRIMARY KEY (`id_estoque`),
                FOREIGN KEY (`id_produto`) REFERENCES `produto` (`id_produto`)
                );
                """,

                # 5. Funcionario (depende de Setor)
                """
                CREATE TABLE IF NOT EXISTS `myindustry`.`funcionario` (
                `id_funcionario` INT NOT NULL AUTO_INCREMENT,
                `nome_funcionario` VARCHAR(100) NOT NULL,
                `cargo` VARCHAR(45) NOT NULL,
                `departamento` VARCHAR(45) NOT NULL,
                `data_contratacao` DATETIME NOT NULL,
                `salario` DECIMAL(10, 2) NOT NULL,
                `contato_emergencia` VARCHAR(200),
                
                PRIMARY KEY (`id_funcionario`)
                
                );
                """,

                # 6. Materia Prima (depende de Equipamento)
                """
                CREATE TABLE IF NOT EXISTS `myindustry`.`materia_prima` (
                `id_materia_prima` INT NOT NULL AUTO_INCREMENT,
                `tipo_materia_prima` VARCHAR(45) NOT NULL,
                `quantidade_recebida` DECIMAL(10, 2) NOT NULL,
                `data_recebimento` DATETIME NOT NULL,
                `qualidade` JSON,
                `id_equipamento` VARCHAR(7) NOT NULL,
                PRIMARY KEY (`id_materia_prima`),
                FOREIGN KEY (`id_equipamento`) REFERENCES `equipamento` (`id_equipamento`)
                );
                """,

                # 7. Evento (depende de Equipamento)
                """
                CREATE TABLE IF NOT EXISTS `myindustry`.`evento` (
                `event_id` VARCHAR(20) NOT NULL,
                `tenant_id` VARCHAR(4) NOT NULL,
                `day` INT NOT NULL,
                `month` INT NOT NULL,
                `year` INT NOT NULL,
                `id_equipamento` VARCHAR(7) NOT NULL,
                PRIMARY KEY (`event_id`),
                FOREIGN KEY (`id_equipamento`) REFERENCES `equipamento` (`id_equipamento`)
                );
                """,

                # 8. Fazenda (depende de Equipamento)
                """
                CREATE TABLE IF NOT EXISTS `myindustry`.`fazenda` (
                `id_fazenda` INT NOT NULL AUTO_INCREMENT,
                `nome_fazenda` VARCHAR(100) NOT NULL,
                `proprietario` VARCHAR(100) NOT NULL,
                `cnpj_cpf` VARCHAR(20) NOT NULL,
                `endereco` TEXT NOT NULL,
                `telefone` VARCHAR(100) NOT NULL,
                `email` VARCHAR(100) NOT NULL,
                `id_equipamento` VARCHAR(7) NOT NULL,
                PRIMARY KEY (`id_fazenda`),
                FOREIGN KEY (`id_equipamento`) REFERENCES `equipamento` (`id_equipamento`)
                );
                """,

                # 9. Manuntencao (depende de Equipamento e Funcionario)
                """
                CREATE TABLE IF NOT EXISTS `myindustry`.`manuntencao` (
                    `idmanuntencao` INT NOT NULL AUTO_INCREMENT,
                    `data_manutencao` DATE NULL,
                    `equipamento_IDequipamento` VARCHAR(7) NOT NULL,
                    `funcionario_idfuncionario` INT NOT NULL,
                    PRIMARY KEY (`idmanuntencao`),
                    INDEX `fk_manuntencao_equipamento1_idx` (`equipamento_IDequipamento` ASC),
                    INDEX `fk_manuntencao_funcionario1_idx` (`funcionario_idfuncionario` ASC),
                    CONSTRAINT `fk_manuntencao_equipamento1`
                        FOREIGN KEY (`equipamento_IDequipamento`)
                        REFERENCES `myindustry`.`equipamento` (`id_equipamento`)
                        ON DELETE NO ACTION
                        ON UPDATE NO ACTION,
                    CONSTRAINT `fk_manuntencao_funcionario1`
                        FOREIGN KEY (`funcionario_idfuncionario`)
                        REFERENCES `myindustry`.`funcionario` (`id_funcionario`)
                        ON DELETE NO ACTION
                        ON UPDATE NO ACTION
                );
                """,
            ]
        for query in create_tables_queries:
            cursor.execute(query)

        connection.commit()
        print("Tables created successfully")

    except Error as e:
        print(f"Error creating tables: {e}")
        connection.rollback()

    finally:
        if connection.is_connected():
            cursor.close()

from datetime import datetime

#def insert_default_setor(connection):
#    try:
#        cursor = connection.cursor()
#        insert_setor_query = """
#        INSERT INTO setor (quantidade_produzida)
#        VALUES (%s)
#        """
#        setor_data = (100,)  # Quantidade produzida padrão
#        cursor.execute(insert_setor_query, setor_data)
#        connection.commit()
#        print("Default setor inserted successfully")
#    except Error as e:
#        print(f"Error inserting default setor: {e}")
#        connection.rollback()
#    finally:
#        if connection.is_connected():
#            cursor.close()

def insert_default_setor(connection):
    try:
        cursor = connection.cursor()
        insert_setor_query = """
        INSERT INTO setor (nome_setor, quantidade_produzida)
        VALUES (%s, %s)
        """
        setor_data = ('Produção Geral', 100)  # Nome do setor e quantidade padrão
        cursor.execute(insert_setor_query, setor_data)
        connection.commit()
        print("Default setor inserted successfully")
    except Error as e:
        print(f"Error inserting default setor: {e}")
        connection.rollback()
    finally:
        if connection.is_connected():
            cursor.close()

def insert_data_to_mysql(connection, df):
    try:
        cursor = connection.cursor()
        
        # Convert timestamp columns to datetime
        date_columns = ['Data_Recebimento', 'Data_Contratacao', 'data_aquisicao']
        for col in date_columns:
            df[col] = pd.to_datetime(df[col]).dt.date
        
        for _, row in df.iterrows():
            # Insert Setor (assumindo que já existe um setor padrão)
            insert_setor_query = """
            INSERT INTO setor (nome_setor, quantidade_produzida)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE 
                nome_setor = VALUES(nome_setor),
                quantidade_produzida = quantidade_produzida + VALUES(quantidade_produzida);
            """
            setor_data = (row['nome_setor'], 100)  # quantidade_produzida padrão
            cursor.execute(insert_setor_query, setor_data)
            
            # Insert Equipamento
            insert_equipamento_query = """
                        INSERT INTO equipamento (id_equipamento, nome_equipamento, data_aquisicao, eqp_status, eventType)
                        VALUES (%s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE
                            nome_equipamento = VALUES(nome_equipamento),
                            data_aquisicao = VALUES(data_aquisicao),
                            eqp_status = VALUES(eqp_status),
                            eventType = VALUES(eventType);
                        """
            equipamento_data = (
            row['id_equipamento'],
            row['nome_equipamento'],
            row['data_aquisicao'],
            row['eqp_status'],
            row['eventType']  # Verifica se 'eventType' existe, caso contrário, define como None
            )
            cursor.execute(insert_equipamento_query, equipamento_data)
            # Insert Produto
            insert_produto_query = """
            INSERT INTO produto (id_produto, id_equipamento, nome_produto, categoria, descricao, unidade_medida, preco_unitario, prazo_validade)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
            """
            produto_data = (
                None,  # id_produto é auto-incremento
                row['id_equipamento'],
                row['Nome_Produto'],
                row['Categoria'],
                row['Descricao'],
                row['Unidade_Medida'],
                row['Preco_Unitario'],
                row['Prazo_Validade']
            )
            cursor.execute(insert_produto_query, produto_data)
            produto_id = cursor.lastrowid

            # Insert Estoque
            insert_estoque_query = """
            INSERT INTO estoque (id_produto, quantidade_estoque)
            VALUES (%s, %s);
            """
            estoque_data = (produto_id, row['quantidade_estoque'])
            cursor.execute(insert_estoque_query, estoque_data)

            # Insert Funcionario
            insert_funcionario_query = """
            INSERT INTO funcionario (nome_funcionario, cargo, departamento, data_contratacao, salario, contato_emergencia)
            VALUES (%s, %s, %s, %s, %s, %s);
            """
            funcionario_data = (
                row['Nome_Funcionario'],
                row['Cargo'],
                row['Departamento'],
                row['Data_Contratacao'],
                row['Salario'],
                row['Contato_Emergencia'],
                
            )
            cursor.execute(insert_funcionario_query, funcionario_data)

            # Insert Materia Prima
            insert_materia_prima_query = """
            INSERT INTO materia_prima (tipo_materia_prima, quantidade_recebida, data_recebimento, qualidade, id_equipamento)
            VALUES (%s, %s, %s, %s, %s);
            """
            materia_prima_data = (
                row['Tipo_MateriaPrima'],
                row['Quantidade_Recebida'],
                row['Data_Recebimento'],
                row['Qualidade'],
                row['id_equipamento']
            )
            cursor.execute(insert_materia_prima_query, materia_prima_data)

            # Insert Evento
            insert_evento_query = """
            INSERT INTO evento (event_id, tenant_id, day, month, year, id_equipamento)
            VALUES (%s, %s, %s, %s, %s, %s);
            """
            evento_data = (
                row['eventId'],
                row['tenantId'],
                row['day'],
                row['month'],
                row['year'],
                row['id_equipamento']
                
            )
            cursor.execute(insert_evento_query, evento_data)

            # Insert Fazenda
            insert_fazenda_query = """
            INSERT INTO fazenda (nome_fazenda, proprietario, cnpj_cpf, endereco, telefone, email, id_equipamento)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
            """
            fazenda_data = (
                row['Nome_Fazenda'],
                row['Proprietario'],
                row['CNPJ_CPF'],
                row['Endereco'],
                row['Telefone'],
                row['Email'],
                row['id_equipamento']
            )
            cursor.execute(insert_fazenda_query, fazenda_data)

        connection.commit()
        print(f"Inserted data from {len(df)} rows into MySQL tables")
    except mysql.connector.Error as e:
        print(f"Error inserting data: {e}")
        connection.rollback()
    finally:
        if connection.is_connected():
            cursor.close()
def main():
    storage_client = get_gcs_client()
    bucket = storage_client.bucket(gcp_bucket_name)
    
    connection = None
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            
            # Criar tabelas
            create_tables(connection)
            insert_default_setor(connection)
            
            # List all blobs in the bucket
            blobs = bucket.list_blobs()
            
            for blob in blobs:
                if blob.name.endswith('.parquet'):
                    print(f"Processing file: {blob.name}")
                    df = read_parquet_from_gcs(bucket, blob.name)
                    insert_data_to_mysql(connection, df)
            
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")
    
    main()
    
    end_time = datetime.now()
    print(f"End time: {end_time}")
    print(f"Elapsed time: {end_time - start_time}")

Start time: 2024-10-19 23:10:41.155190
Connected to MySQL database
Tables created successfully
Default setor inserted successfully
Processing file: bwfg_00cd_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_00en_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_00l8_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_00o0_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_0117_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_0137_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_01dl_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_01lc_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_01pb_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_0201_.parquet
Inserted data from 1 rows into MySQL tables
Processing file: bwfg_0209_.parquet
Inserted data from 1 rows into My

In [ ]:
eventType
quantidade_produzida

In [ ]:
def insert_data_to_mysql(connection, df):
    try:
        cursor = connection.cursor()

        # Convert timestamp columns to datetime
        date_columns = ['Data_Recebimento', 'Data_Contratacao']
        for col in date_columns:
            df[col] = pd.to_datetime(df[col]).dt.date

        # Iterar sobre as linhas do DataFrame para inserir dados
        for _, row in df.iterrows():
            
            # Inserir Fornecedor
            insert_fornecedor_query = """
            INSERT INTO fornecedor (cnpj, nome, email, telefone)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            nome = VALUES(nome),
            email = VALUES(email),
            telefone = VALUES(telefone)
            """
            fornecedor_data = (
                row['CNPJ_CPF'],
                row['Nome_Fazenda'],
                row['Email'],
                row['Telefone']
            )
            cursor.execute(insert_fornecedor_query, fornecedor_data)
            
            # Inserir Estoque
            insert_estoque_query = """
            INSERT INTO estoque (quantidade_disponivel)
            VALUES (%s)
            """
            estoque_data = (
                row['quantidade_estoque'],  # Aqui assumimos que você tem uma coluna 'quantidade_estoque' no Parquet
            )
            cursor.execute(insert_estoque_query, estoque_data)
            
            # Obtenha o último ID do estoque inserido
            estoque_id = cursor.lastrowid
            
            # Inserir Materia_Prima
            insert_materia_prima_query = """
            INSERT INTO mateiraPrima (data_recebimento, quantidade, tipo, descricao)
            VALUES (%s, %s, %s, %s)
            """
            materia_prima_data = (
                row['Data_Recebimento'],
                row['Quantidade_Recebida'],
                row['Tipo_MateriaPrima'],
                'Descrição da matéria prima'
            )
            cursor.execute(insert_materia_prima_query, materia_prima_data)

            # Inserir Produto com o estoque_id correspondente
            insert_produto_query = """
            INSERT INTO produto (nome_produto, categoria, validade_lote, setor_idsetor, estoque_idestoque)
            VALUES (%s, %s, %s, %s, %s)
            """
            validade_lote = row['Data_Recebimento'] + pd.Timedelta(days=int(row['Prazo_Validade']))
            produto_data = (
                row['Nome_Produto'],
                row['Categoria'],
                validade_lote,
                1,  # setor_idsetor (você pode ajustar isso conforme necessário)
                estoque_id  # Use o estoque_id que acabamos de inserir
            )
            cursor.execute(insert_produto_query, produto_data)
            
            # Inserir Funcionario
            insert_funcionario_query = """
            INSERT INTO funcionario (cargo, nome, data_contratacao, contato, salario, setor_idsetor)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            funcionario_data = (
                row['Cargo'],
                row['Nome_Funcionario'],
                row['Data_Contratacao'],
                row['Contato_Emergencia'].split(',')[1].strip(),
                row['Salario'],
                1  # setor_idsetor (você pode ajustar isso)
            )
            cursor.execute(insert_funcionario_query, funcionario_data)

        connection.commit()
        print(f"Inserted data from {len(df)} rows into MySQL tables")

    except Error as e:
        print(f"Error inserting data: {e}")
        connection.rollback()

    finally:
        if connection.is_connected():
            cursor.close()